# DDW HW 4 - Social Network Analysis
For CTU in Prague, by Antonín Karola

## Python code

In [1]:
import numpy as np
import csv

# read file
lines = []

with open('casts.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=';')
    for row in readCSV:
        lines.append(row[1:3])

# use numpy array for easy array slicing
data = np.array(lines)

# for all rows in data, extract movies (column 0)/actors (col 1); remove duplicates using np.unique + order
movies = np.unique(list(filter(None, data[:,0])))
actors = np.unique(list(filter(None, data[:,1])))

In [2]:
import networkx as nx

# create graph, create a node for each actor
G=nx.Graph()

for actor in actors:
    if actor:
        G.add_node(actor)

In [3]:
# empty dictionary: [movie] = [actor1, actor2, ...]
moviesWithActors = dict(zip(movies, [None] * len(movies)))

# map actors to the movie dictionary above
for row in data:
    movie = row[0]; actor = row[1]
    if movie:
        if not moviesWithActors[movie]:
            moviesWithActors[movie] = [actor]
        else:
            moviesWithActors[movie].append(actor)

In [4]:
# create edges between actors that appeared in the same movie
for movie in moviesWithActors:
    movieActors = moviesWithActors[movie]
    for fromActor in movieActors:
        i = 0
        for toActor in movieActors[i:]:
            if fromActor and toActor:
                G.add_edge(fromActor, toActor)
            i += 1

In [ ]:
# export graph to gephi
nx.write_gexf(G, "export.gexf")
#nx.draw(G)

<hr>
## Report

## Provide general statistics about the dataset

### number of nodes and edges

In [5]:
# nodes
n = len(actors)
n

16614

In [28]:
G.number_of_edges()

172188

### density

In [7]:
possibleEdges = n*(n-1)/2
edges/possibleEdges

0.00282623300911202

### number of components

In [8]:
nx.number_connected_components(G)

640

## Provide list of top key players using different centralities
Identify key players using centrality measures

#### Degree Centrality

In [9]:
import operator
for key, value in sorted(nx.degree_centrality(G).items(), key=operator.itemgetter(1), reverse=True)[0:10]:
    print("%s:\t%s" % (key, value))

s a:	0.19930175164028172
Humphrey Bogart:	0.026003732017095046
James Stewart:	0.022632877866730874
Gary Cooper:	0.02239210257027629
John Gielgud:	0.02239210257027629
John Carradine:	0.022211521097935352
Peter Lorre:	0.02179016432913983
C.Aubrey Smith:	0.020405706374525975
Henry Fonda:	0.019623186661048578
Burt Lancaster:	0.018900860771684826


#### Closeness Centrality

In [10]:
for key, value in sorted(nx.closeness_centrality(G).items(), key=operator.itemgetter(1), reverse=True)[0:10]:
    print("%s:\t%s" % (key, value))

s a:	0.47334397258147604
Charlton Heston:	0.36832541164881627
John Gielgud:	0.366879133608347
John Carradine:	0.3665439581315982
Burt Lancaster:	0.3663765999469707
Henry Fonda:	0.3657184972071195
James Stewart:	0.3650920657805791
Humphrey Bogart:	0.36309023012578007
Robert Mitchum:	0.3616942846566501
Peter Lorre:	0.3606615008282145


#### Betweenness Centrality

In [11]:
for key, value in sorted(nx.betweenness_centrality(G).items(), key=operator.itemgetter(1), reverse=True)[0:10]:
    print("%s:\t%s" % (key, value))

s a:	0.34429277261378943
Humphrey Bogart:	0.008273686145191887
Vincent Price:	0.007787663365423353
John Carradine:	0.007038387205951536
Jack Nicholson:	0.006673278925380563
Burt Lancaster:	0.006558833266695104
Charlton Heston:	0.006133041525240112
John Gielgud:	0.005786799065031964
Max vonSydow:	0.00575501153303901
Robert deNiro:	0.005684354165728045


#### Eigenvector Centrality

In [13]:
for key, value in sorted(nx.eigenvector_centrality(G).items(), key=operator.itemgetter(1), reverse=True)[0:10]:
    print("%s:\t%s" % (key, value))

s a:	0.32574473945821847
C.Aubrey Smith:	0.08707091404834201
John Carradine:	0.08561343912976806
James Stewart:	0.08367120801307142
John Gielgud:	0.08140384894958089
Peter Lorre:	0.07888536941271428
Gary Cooper:	0.07783581126958793
Basil Rathbone:	0.07529313121494362
Humphrey Bogart:	0.07442858275700319
Henry Fonda:	0.07438018567991815


In [ ]:
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
plt.figure(figsize=(18, 18))
pos = nx.random_layout(G)
nx.draw(G, pos, font_size=8, labels={v: str(v) for v in G},
        cmap=plt.get_cmap("bwr"), node_color=[nx.degree_centrality(G)[k] for k in nx.degree_centrality(G)])

## Describe top clusters/communities
Identify clusters/communities in graph

In [ ]:
from networkx.algorithms import community
communities_generator = community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
sorted(map(sorted, next_level_communities))

Networkx algorithm (above) ran too long, I used community detection in Gephi.

Modularity Report from Gephi:
![Modularity Report from Gephi](Gephi/communities/communities-size-distribution.png)

## Describe „Kevin Bacon“ numbers
Compute „Kevin Bacon“ number for each actor with selected key player
- e.g. top actors with the highest/lowest number
- average number

In [24]:
shortestPathLengths = nx.single_source_shortest_path_length(G, 'Kevin Bacon')

top 10 hightest (closest actors to Kevin Bacon)

In [25]:
for key, value in sorted(shortestPathLengths.items(), key=operator.itemgetter(1), reverse=True)[0:10]:
    print("%s:\t%s" % (key, value))

Robert Castle:	6
Paredes:	6
Antonia SanJuan:	6
Elisa Touati:	6
Marbel Verdu:	6
Maria deMederios:	6
Barbara Dennek:	6
Jacqueline Lecomte:	6
Henri Piccoli:	6
Ann Ruymen:	5


top 10 lowest (most distant actors from Kevin Bacon)

In [26]:
for key, value in sorted(shortestPathLengths.items(), key=operator.itemgetter(1), reverse=False)[0:10]:
    print("%s:\t%s" % (key, value))

Kevin Bacon:	0
Tom Cruise:	1
Jack Nicholson:	1
Demi Moore:	1
s a:	1
J.A. Preston:	1
Michael deLeon:	1
Kiefer Sutherland:	1
Evan Rachel:	1
Mary Stuart Masterson:	1


## Visualise important aspects of the analysis
Insert visualisations from networkx/Gephi (images/screenshots)
- for visualisation you can use only subset of actors/movies e.g. movies from specific category, movies with max 5 actors etc.

<h1 style="color:red">TODO</h1>

## Insert a file with the graph exported to a GEXF format
All computed values should be present as attributes
- see `export.gefx` on the GitHub

## Comments
**issues during the design/implementation**
- The hardest part was to find out how to process the CSV file to make a graph from the data - how to map a list of actors in a movie to nodes with edges (result is 3 nested for loops)
- Centralities took too long to calculate (like 20 min, due to the huge data set)
- The same goes for finding communities in the graph (`girvan_newman` method internally uses `networkx.edge_betweenness_centrality()`)


**ideas for extensions/improvements/future work**
- filter invalid values from data (e.g. actor name "s a"...)
 
